In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
# Get basic players information for all players
base_url = "https://sofifa.com/players?offset="
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special']
data = pd.DataFrame(columns = columns)

In [3]:
for offset in range(0, 300):
    url = base_url + str(offset * 61)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    table_body = soup.find('tbody')
    for row in table_body.find_all('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        nationality = td[1].find('a').get('title')
        flag_img = td[1].find('img').get('data-src')
        name = td[1].findAll('a')[1].text
        age = td[2].find('div').text.strip()
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        club_logo = td[5].find('img').get('data-src')
        value = td[6].text.strip()
        wage = td[7].text.strip()
        special = td[8].text.strip()
        player_data = pd.DataFrame([[pid, name, age, picture, nationality, flag_img, overall, potential, club, club_logo, value, wage, special]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
data = data.drop_duplicates()

In [4]:
data.head(10)

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2217
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,92,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€108M,€290K,2150
3,200389,J. Oblak,25,https://cdn.sofifa.org/players/4/19/200389.png,Slovenia,https://cdn.sofifa.org/flags/44.png,91,93,Atlético Madrid,https://cdn.sofifa.org/teams/2/light/240.png,€75.5M,€120K,1330
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2279
5,183277,E. Hazard,27,https://cdn.sofifa.org/players/4/19/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,91,Chelsea,https://cdn.sofifa.org/teams/2/light/5.png,€93M,€340K,2143
6,177003,L. Modrić,32,https://cdn.sofifa.org/players/4/19/177003.png,Croatia,https://cdn.sofifa.org/flags/10.png,91,91,Real Madrid,https://cdn.sofifa.org/teams/2/light/243.png,€67M,€420K,2284
7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€80M,€455K,2349
8,202126,H. Kane,24,https://cdn.sofifa.org/players/4/19/202126.png,England,https://cdn.sofifa.org/flags/14.png,90,92,Tottenham Hotspur,https://cdn.sofifa.org/teams/2/light/18.png,€96.5M,€210K,2177
9,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,90,92,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€62.5M,€215K,1451


In [ ]:
player_data_url = 'https://sofifa.com/player/'

# Generate the data for 2019
for id in data.ID:
    url = player_data_url + str(id)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    skill_map = {}
    columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
    for column in columns:
        skills = column.findAll('li')
        for skill in skills:
            if(skill.find('label') != None):
                label = skill.find('label').text
                value = skill.text.replace(label, '').strip()
                skill_map[label] = value
    meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
    length = len(meta_data)
    weight = meta_data[length - 1]
    height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
    skill_map["Height"] = height
    skill_map['Weight'] = weight
    if('Position' in skill_map.keys()):
        if skill_map['Position'] in ('', 'RES', 'SUB'):
            skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
        if(skill_map['Position'] != 'GK'):
            card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
            for c_row in card_rows:
                attributes = c_row.findAll('div', {'class': re.compile('column col-sm-2 text-center')})
                for attribute in attributes:
                    if(attribute.find('div')):
                        name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                        value = attribute.text.replace(name, '').strip()
                        skill_map[str(name)] = value
    sections = soup.find('article').findAll('div', {'class': 'mb-2'})[1:3]
    first = sections[0].findAll('div', {'class': 'column col-4'})
    second = sections[1].findAll('div', {'class': 'column col-4'})[:-1]
    sections = first + second
    for section in sections:
        items = section.find('ul').findAll('li')
        for item in items:
            value = int(re.findall(r'\d+', item.text)[0])
            name = ''.join(re.findall('[a-zA-Z]*', item.text))
            skill_map[str(name)] = value
    for key, value in skill_map.items():
        detailed_data.loc[detailed_data.ID == id, key] = value

In [7]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2019 = pd.DataFrame(index = range(0, data.count()[0]), columns = detailed_columns)
detailed_data_2019.ID = data.ID.values

In [8]:
player_data_url = 'https://sofifa.com/player/'
count = 0
# Generate the data for 2019 

for id in data.ID:
    while count < 5:
        url = player_data_url + str(id)
        source_code = requests.get(url)
        redirect_url = source_code.url
        if redirect_url == url:
            plain_text = source_code.text
            soup = BeautifulSoup(plain_text, 'html.parser')
            skill_map = {}
            overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
            skill_map['Overall'] = int(overall)
            first_find = soup.find('div', {'class': 'column col-4 text-center'})
            second_find = first_find.find_next('div').find('span').text
            skill_map['Expert Potential'] = int(second_find)
            columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
            for column in columns:
                skills = column.find_all('li')
                for skill in skills:
                    if(skill.find('label') != None):
                        label = skill.find('label').text
                        value = skill.text.replace(label, '').strip()
                        skill_map[label] = value
            meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
            length = len(meta_data)
            weight = meta_data[length - 1]
            height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
            skill_map["Height"] = height
            skill_map['Weight'] = weight
            if('Position' in skill_map.keys()):
                if skill_map['Position'] in ('', 'RES', 'SUB'):
                    skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
                if(skill_map['Position'] != 'GK'):
                    card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                    for c_row in card_rows:
                        attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                        for attribute in attributes:
                            if(attribute.find('div')):
                                name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                                value = attribute.text.replace(name, '').strip()
                                skill_map[str(name)] = value
            sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
            first = sections[0].find_all('div', {'class': 'column col-4'})
            second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
            sections = first + second
            for section in sections:
                items = section.find('ul').find_all('li')
                for item in items:
                    value = int(re.findall(r'\d+', item.text)[0])
                    name = ''.join(re.findall('[a-zA-Z]*', item.text))
                    skill_map[str(name)] = value
            for key, value in skill_map.items():
                detailed_data_2019.loc[detailed_data_2019.ID == id, key] = value
            count += 1

detailed_data_2019['Year'] = int(2019)

KeyboardInterrupt: 

In [15]:
pd.set_option('display.max_columns', 500)

In [16]:
detailed_data_2019

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Release Clause
0,94,94,Left,5,4,4,Medium/ Low,Messi,Yes,CF,10,"Jul 1, 2004",NaN,2021,5'7,159lbs,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,91+2,85+2,85+2,85+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,48+2,48+2,48+2,59+2,86,95,70,92,86,97,93,94,89,96,91,86,93,95,95,85,68,72,66,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,158023,€226.5M
1,94,94,Right,5,4,5,High/ Low,C. Ronaldo,Yes,LW,7,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,20801,€127.1M
2,92,92,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,66+3,60+3,60+3,60+3,66+3,61+3,47+3,47+3,47+3,61+3,83,87,62,84,84,96,88,87,80,95,94,90,96,94,84,80,61,81,49,82,56,36,89,88,81,94,27,24,33,9,9,15,15,11,190871,€199.8M
3,91,93,Right,3,3,1,Medium/ Medium,Normal,Yes,GK,1,"Jul 16, 2014",NaN,2023,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,29,13,12,13,14,26,16,43,60,67,88,49,22,76,41,78,12,34,19,11,65,11,70,27,12,18,86,92,78,90,89,200389,€160.4M
4,91,92,Right,4,5,4,High/ High,Normal,Yes,RCM,7,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,86+3,86+3,86+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93,82,55,92,82,86,85,83,91,91,78,76,79,91,77,91,63,89,75,90,76,61,87,94,79,90,68,58,51,15,13,5,10,13,192985,€196.4M
5,91,91,Right,4,4,4,High/ Medium,Normal,Yes,LF,10,"Jul 1, 2012",NaN,2020,5'8,168lbs,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3,81,84,61,89,80,95,83,79,83,94,94,88,95,90,94,82,56,84,66,80,54,41,87,89,86,91,34,27,22,11,12,6,8,8,183277,€172.1M
6,91,91,Right,4,4,4,High/ High,Lean,Yes,RCM,10,"Aug 1, 2012",NaN,2020,5'8,146lbs,77+3,77+3,77+3,85+3,84+3,84+3,84+3,85+3,87+3,87+3,87+3,86+3,88+3,88+3,88+3,86+3,82+3,82+3,82+3,82+3,82+3,80+3,72+3,72+3,72+3,80+3,86,72,55,93,76,90,85,78,88,92,78,72,92,90,94,79,68,89,58,82,62,83,79,92,82,92,68,76,73,13,9,7,14,9,177003,€137.4M
7,91,91,Right,5,4,3,High/ Medium,Normal,Yes,RS,9,"Jul 11, 2014",NaN,2021,6'0,190lbs,87+5,87+5,87+5,85+5,86+5,86+5,86+5,85+5,85+5,85+5,85+5,84+5,79+5,79+5,79+5,84+5,69+5,68+5,68+5,68+5,69+5,66+5,63+5,63+5,63+5,66+5,77,92,82,82,90,86,86,84,69,88,83,73,82,92,83,86,69,90,83,85,87,41,92,84,85,85,62,45,38,27,25,31,33,37,176580,€164M
8,90,92,Right,3,4,3,High/ High,Normal,Yes,ST,9,"Jul 1, 2010",NaN,2024,6'2,196lbs,86+3,86+3,86+3,82+3,84+3,84+3,84+3,82+3,83+3,83+3,83+3,82+3,79+3,79+3,79+3,82+3,65+3,66+3,66+3,66+3,65+3,62+3,60+3,60+3,60+3,62+3,75,94,86,82,85,80,78,68,83,84,68,72,71,91,73,88,79,89,84,86,78,35,93,81,90,91,56,36,38,8,10,11,14,11,202126,€185.8M
9,90,92,Right,4,3,1,Medium/ Medium,Lean,Yes,GK,1,"Jul 1, 2011",NaN,2020,6'4,181lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,13,21,50,13,18,21,19,47,38,57,58,62,88,43,31,67,43,64,12,38,30,12,65,29,70,25,21,13,90,84,83,86,93,193080,€123.4M


In [18]:
detailed_data_2019_clean = detailed_data_2019.drop('Loaned From', axis=1)

In [19]:
detailed_data_2019_clean = detailed_data_2019_clean.dropna()

In [37]:
detailed_data_2019_clean['Year'] = int(2019)

In [38]:
detailed_data_2019_clean.head()

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Release Clause,Year
0,94,94,Left,5,4,4,Medium/ Low,Messi,Yes,CF,10,"Jul 1, 2004",2021,5'7,159lbs,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,91+2,85+2,85+2,85+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,48+2,48+2,48+2,59+2,86,95,70,92,86,97,93,94,89,96,91,86,93,95,95,85,68,72,66,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,158023,€226.5M,2019
1,94,94,Right,5,4,5,High/ Low,C. Ronaldo,Yes,LW,7,"Jul 10, 2018",2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,20801,€127.1M,2019
2,92,92,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",2022,5'9,150lbs,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,89+3,89+3,89+3,89+3,81+3,81+3,81+3,89+3,66+3,60+3,60+3,60+3,66+3,61+3,47+3,47+3,47+3,61+3,83,87,62,84,84,96,88,87,80,95,94,90,96,94,84,80,61,81,49,82,56,36,89,88,81,94,27,24,33,9,9,15,15,11,190871,€199.8M,2019
4,91,92,Right,4,5,4,High/ High,Normal,Yes,RCM,7,"Aug 30, 2015",2023,5'11,154lbs,82+3,82+3,82+3,87+3,86+3,86+3,86+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93,82,55,92,82,86,85,83,91,91,78,76,79,91,77,91,63,89,75,90,76,61,87,94,79,90,68,58,51,15,13,5,10,13,192985,€196.4M,2019
5,91,91,Right,4,4,4,High/ Medium,Normal,Yes,LF,10,"Jul 1, 2012",2020,5'8,168lbs,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3,81,84,61,89,80,95,83,79,83,94,94,88,95,90,94,82,56,84,66,80,54,41,87,89,86,91,34,27,22,11,12,6,8,8,183277,€172.1M,2019


In [39]:
detailed_data_2019_clean.to_csv('2019_data.csv', encoding='utf-8-sig')

In [22]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2018 = pd.DataFrame(index = range(0, data.count()[0]), columns = detailed_columns)
detailed_data_2018.ID = data.ID.values

In [29]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2018

for id in data.ID[:2000]:
    url = player_data_url + str(id) + '/18/159201'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2018.loc[detailed_data_2018.ID == id, key] = value

detailed_data_2018['Year'] = int(2018)

In [34]:
detailed_data_2018_clean = detailed_data_2018.drop('Loaned From', axis=1)

In [35]:
detailed_data_2018_clean = detailed_data_2018_clean.dropna()

In [41]:
detailed_data_2018_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Release Clause,Year
0,94,94,Left,5,4,4,Medium/ Medium,Messi,Yes,RW,10,"Jul 1, 2004",2021,5'7,159lbs,89+4,89+4,89+4,92+4,93+4,93+4,93+4,92+4,93+4,93+4,93+4,91+4,84+4,84+4,84+4,91+4,63+4,60+4,60+4,60+4,63+4,58+4,46+4,46+4,46+4,58+4,77,95,71,88,86,97,90,92,87,96,92,87,90,95,95,85,67,73,59,88,48,22,93,92,75,97,13,28,26,6,11,15,14,8,158023,€242.9M,2018
1,94,94,Right,5,4,5,High/ Low,C. Ronaldo,Yes,LS,7,"Jul 1, 2009",2021,6'2,183lbs,92+3,92+3,92+3,91+3,92+3,92+3,92+3,91+3,90+3,90+3,90+3,90+3,83+3,83+3,83+3,90+3,66+3,62+3,62+3,62+3,66+3,62+3,53+3,53+3,53+3,62+3,85,94,88,83,88,91,81,76,77,93,89,91,89,96,63,94,95,92,80,92,63,29,95,85,85,95,22,31,23,7,11,15,14,11,20801,€195.8M,2018
2,92,93,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",2022,5'9,150lbs,85+3,85+3,85+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,80+3,80+3,80+3,88+3,64+3,60+3,60+3,60+3,64+3,60+3,47+3,47+3,47+3,60+3,75,88,62,82,83,96,82,84,75,95,94,90,96,88,82,80,61,78,53,78,56,36,90,83,81,92,21,24,33,9,9,15,15,11,190871,€230M,2018
4,91,92,Right,4,5,4,High/ High,Normal,Yes,RCM,7,"Aug 30, 2015",2023,5'11,150lbs,83+3,83+3,83+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,75+3,74+3,74+3,74+3,75+3,71+3,63+3,63+3,63+3,71+3,92,83,54,91,82,86,85,83,87,90,78,76,82,90,79,89,65,88,73,88,73,60,86,92,77,86,40,58,51,15,13,5,10,13,192985,€201.2M,2018
5,91,91,Right,4,4,4,High/ Medium,Normal,Yes,LF,10,"Jul 1, 2012",2020,5'8,168lbs,84+2,84+2,84+2,89+2,89+2,89+2,89+2,89+2,90+2,90+2,90+2,89+2,83+2,83+2,83+2,89+2,66+2,63+2,63+2,63+2,66+2,60+2,49+2,49+2,49+2,60+2,81,84,60,89,80,94,83,79,83,93,93,88,93,87,93,79,59,81,67,80,54,41,87,89,86,89,25,27,22,11,12,6,8,8,183277,€176.7M,2018
6,89,89,Right,4,4,4,High/ Medium,Lean,Yes,RCM,10,"Aug 1, 2012",2020,5'8,146lbs,77+3,77+3,77+3,84+3,83+3,83+3,83+3,84+3,86+3,86+3,86+3,84+3,87+3,87+3,87+3,84+3,81+3,81+3,81+3,81+3,81+3,79+3,73+3,73+3,73+3,79+3,78,71,55,92,74,86,79,77,84,92,75,71,93,88,94,73,67,82,58,82,62,78,79,90,80,84,66,80,73,13,9,7,14,9,177003,€116.9M,2018
7,92,92,Right,5,4,4,High/ Medium,Normal,Yes,LS,9,"Jul 11, 2014",2021,6'0,190lbs,88+4,88+4,88+4,87+4,89+4,89+4,89+4,87+4,87+4,87+4,87+4,86+4,81+4,81+4,81+4,86+4,68+4,66+4,66+4,66+4,68+4,65+4,60+4,60+4,60+4,65+4,77,94,77,83,88,86,86,84,64,91,88,77,86,93,78,87,69,89,83,86,87,41,92,84,85,85,30,45,38,27,25,31,33,37,176580,€198.9M,2018
8,88,91,Right,3,4,3,High/ High,Normal,Yes,RS,9,"Jul 1, 2010",2024,6'2,190lbs,86+2,86+2,86+2,83+2,85+2,85+2,85+2,83+2,83+2,83+2,83+2,82+2,79+2,79+2,79+2,82+2,65+2,65+2,65+2,65+2,65+2,62+2,58+2,58+2,58+2,62+2,75,92,83,79,77,80,75,68,80,84,72,75,73,90,62,87,70,88,87,85,76,35,92,80,86,89,41,36,38,8,10,11,14,11,202126,€147.3M,2018
10,91,91,Right,4,4,3,High/ Medium,Normal,Yes,LS,9,"Jul 1, 2014",2021,6'1,174lbs,88+3,88+3,88+3,84+3,87+3,87+3,87+3,84+3,85+3,85+3,85+3,82+3,79+3,79+3,79+3,82+3,62+3,63+3,63+3,63+3,62+3,59+3,57+3,57+3,57+3,59+3,62,91,85,85,89,85,77,84,65,89,79,79,78,91,80,88,84,79,84,84,80,39,91,78,88,87,25,42,19,15,6,12,8,10,188545,€151.8M,2018
11,88,88,Right,3,3,2,Medium/ High,Lean,Yes,RCB,3,"Aug 1, 2010",2019,6'2,174lbs,65+2,65+2,65+2,62+2,63+2,63+2,63+2,62+2,63+2,63+2,63+2,64+2,69+2,69+2,69+2,64+2,77+2,81+2,81+2,81+2,77+2,80+2,86+2,86+2,86+2,80+2,55,42,92,79,47,53,49,51,70,76,64,70,63,85,58,67,89,67,80,43,88,88,48,52,50,82,87,86,89,6,8,15,5,15,182493,€82M,2018


In [40]:
detailed_data_2018_clean.to_csv('2018_data.csv', encoding='utf-8-sig')

In [18]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2017 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2017.ID = id_data.ID.values

In [3]:
id_data = pd.read_csv('id_data.csv')

In [21]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2017

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/17/158837'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2017.loc[detailed_data_2017.ID == id, key] = value

detailed_data_2017['Year'] = int(2017)

In [22]:
detailed_data_2017_clean = detailed_data_2017.drop('Loaned From', axis=1)

In [23]:
detailed_data_2017_clean = detailed_data_2017_clean.dropna()

In [24]:
detailed_data_2017_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,...,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,93,93,Left,5,4,4,Medium/ Medium,Messi,Yes,RW,...,13,28,26,6,11,15,14,8,158023,2017
1,94,94,Right,5,4,5,High/ Low,C. Ronaldo,Yes,LS,...,22,31,23,7,11,15,14,11,20801,2017
2,92,94,Right,5,5,5,High/ Medium,Neymar,Yes,LW,...,21,24,33,9,9,15,15,11,190871,2017
4,88,91,Right,4,4,4,High/ High,Normal,Yes,RCM,...,30,39,40,15,13,5,10,13,192985,2017
5,89,90,Right,4,4,4,High/ Medium,Normal,Yes,LF,...,25,27,22,11,12,6,8,8,183277,2017
6,89,89,Right,4,4,4,High/ Medium,Lean,Yes,RCM,...,66,80,73,13,9,7,14,9,177003,2017
7,92,92,Right,5,4,4,High/ Medium,Normal,Yes,LS,...,30,45,38,27,25,31,33,37,176580,2017
8,85,90,Right,3,3,3,High/ High,Normal,Yes,ST,...,41,36,38,8,10,11,14,11,202126,2017
10,90,90,Right,4,4,3,High/ Medium,Normal,Yes,LS,...,25,42,19,15,6,12,8,10,188545,2017
11,88,88,Right,3,3,2,Medium/ High,Lean,Yes,LCB,...,87,86,89,6,8,15,5,15,182493,2017


In [25]:
detailed_data_2017_clean.to_csv('2017_data.csv', encoding='utf-8-sig')

In [26]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2016 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2016.ID = id_data.ID.values

In [27]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2016

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/16/158467'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2016.loc[detailed_data_2016.ID == id, key] = value

detailed_data_2016['Year'] = int(2016)

In [32]:
pd.set_option('display.max_columns', 500)

In [39]:
detailed_data_2016_clean = detailed_data_2016.drop('Loaned From', axis=1)

In [40]:
detailed_data_2016_clean = detailed_data_2016_clean.drop('Composure', axis=1)

In [41]:
detailed_data_2016_clean = detailed_data_2016_clean.dropna()

In [42]:
detailed_data_2016_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,94,94,Left,5,4,4,Medium/ Low,Messi,Yes,RW,10,"Jul 1, 2004",2018,5'7,159lbs,87+3,87+3,87+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,90+3,82+3,82+3,82+3,90+3,62+3,57+3,57+3,57+3,62+3,56+3,44+3,44+3,44+3,56+3,80,93,71,88,85,96,89,90,79,96,95,90,92,92,95,80,68,75,59,88,48,22,90,90,74,13,23,21,6,11,15,14,8,158023,2016
1,93,93,Right,5,4,5,High/ Low,C. Ronaldo,Yes,LS,7,"Jul 1, 2009",2018,6'1,176lbs,91+3,91+3,91+3,90+3,91+3,91+3,91+3,90+3,88+3,88+3,88+3,88+3,80+3,80+3,80+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,52+3,52+3,52+3,60+3,82,95,86,81,87,93,88,77,72,91,91,93,90,92,62,94,94,90,79,93,62,29,93,81,85,22,31,23,7,11,15,14,11,20801,2016
2,90,94,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Jul 1, 2013",2018,5'9,150lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,86+3,86+3,86+3,85+3,77+3,77+3,77+3,85+3,63+3,58+3,58+3,58+3,63+3,58+3,45+3,45+3,45+3,58+3,72,88,62,78,83,94,78,79,74,93,91,90,92,86,84,78,61,79,45,73,56,36,89,79,81,21,24,33,9,9,15,15,11,190871,2016
4,86,89,Right,3,5,4,High/ Medium,Normal,Yes,CAM,7,"Aug 30, 2015",2021,5'11,168lbs,80+1,80+1,80+1,84+1,84+1,84+1,84+1,84+1,85+1,85+1,85+1,85+1,82+1,82+1,82+1,85+1,69+1,67+1,67+1,67+1,69+1,64+1,55+1,55+1,55+1,64+1,90,82,53,88,82,85,78,77,82,86,76,78,78,87,75,84,65,85,73,85,68,52,83,88,77,30,39,40,15,13,5,10,13,192985,2016
5,88,90,Right,4,4,4,High/ Medium,Normal,Yes,LM,10,"Jul 1, 2012",2020,5'8,163lbs,81+2,81+2,81+2,87+2,86+2,86+2,86+2,87+2,87+2,87+2,87+2,86+2,80+2,80+2,80+2,86+2,64+2,61+2,61+2,61+2,64+2,58+2,47+2,47+2,47+2,58+2,80,81,57,85,79,94,82,79,82,90,93,87,92,86,90,79,59,74,63,82,54,41,84,86,86,25,27,22,11,12,6,8,8,183277,2016
6,87,87,Right,4,4,4,High/ Medium,Lean,Yes,RCM,19,"Aug 1, 2012",2018,5'9,143lbs,76+2,76+2,76+2,83+2,82+2,82+2,82+2,83+2,85+2,85+2,85+2,83+2,85+2,85+2,85+2,83+2,80+2,79+2,79+2,79+2,80+2,78+2,71+2,71+2,71+2,78+2,78,71,55,86,75,86,79,77,83,92,78,74,93,88,94,73,67,87,61,82,62,73,79,89,80,69,75,73,13,9,7,14,9,177003,2016
7,90,90,Right,5,4,4,High/ Medium,Normal,Yes,RS,9,"Jul 11, 2014",2019,6'0,187lbs,87+3,87+3,87+3,87+3,88+3,88+3,88+3,87+3,86+3,86+3,86+3,85+3,80+3,80+3,80+3,85+3,67+3,65+3,65+3,65+3,67+3,64+3,58+3,58+3,58+3,64+3,77,90,77,82,87,88,86,84,64,91,88,78,86,91,60,88,69,88,76,85,78,41,91,84,85,30,45,38,27,25,31,33,37,176580,2016
8,82,88,Right,3,3,3,High/ High,Normal,Yes,ST,9,"Jul 1, 2010",2020,6'2,143lbs,81+1,81+1,81+1,78+1,79+1,79+1,79+1,78+1,77+1,77+1,77+1,77+1,72+1,72+1,72+1,77+1,62+1,60+1,60+1,60+1,62+1,59+1,56+1,56+1,56+1,59+1,69,85,81,75,70,77,71,71,63,81,69,77,74,84,62,82,67,84,82,78,74,35,84,72,82,41,36,38,8,10,11,14,11,202126,2016
10,88,88,Right,4,4,3,Medium/ Medium,Normal,Yes,LS,9,"Jul 1, 2014",2019,6'1,174lbs,86+2,86+2,86+2,82+2,85+2,85+2,85+2,82+2,83+2,83+2,83+2,81+2,77+2,77+2,77+2,81+2,61+2,62+2,62+2,62+2,61+2,58+2,56+2,56+2,56+2,58+2,62,89,83,82,84,85,77,68,65,87,79,81,78,88,78,85,83,79,82,82,80,39,88,78,77,25,42,19,15,6,12,8,10,188545,2016
11,86,86,Right,3,3,2,Medium/ High,Lean,Yes,LCB,3,"Aug 1, 2010",2019,6'1,161lbs,64+1,64+1,64+1,61+1,62+1,62+1,62+1,61+1,62+1,62+1,62+1,63+1,68+1,68+1,68+1,63+1,76+1,79+1,79+1,79+1,76+1,79+1,85+1,85+1,85+1,79+1,55,42,92,77,47,53,49,51,69,73,68,69,62,83,58,67,88,73,78,43,84,86,48,52,50,86,86,89,6,8,15,5,15,182493,2016


In [43]:
detailed_data_2016_clean.to_csv('2016_data.csv', encoding='utf-8-sig')

In [44]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2015 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2015.ID = id_data.ID.values

In [54]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2015

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/15/158102'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2015.loc[detailed_data_2015.ID == id, key] = value

detailed_data_2015['Year'] = int(2015)

In [61]:
detailed_data_2015_clean = detailed_data_2015.drop('Loaned From', axis=1)

In [62]:
detailed_data_2015_clean = detailed_data_2015_clean.drop('Composure', axis=1)

In [64]:
detailed_data_2015_clean = detailed_data_2015_clean.dropna()

In [65]:
detailed_data_2015_clean # 2015没处理完

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,93,93,Left,5,4,4,Medium/ Low,Normal,Yes,RW,10,"Jul 1, 2004",2018,5'7,159lbs,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,74,25,21,20,6,11,15,14,8,158023,2015
1,92,92,Right,5,4,5,High/ Low,Normal,Yes,LS,7,"Jul 1, 2009",2018,6'1,176lbs,91+3,91+3,91+3,89+3,91+3,91+3,91+3,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,22,31,23,7,11,15,14,11,20801,2015
2,86,91,Right,5,5,5,High/ Medium,Lean,Yes,LW,10,"Jul 1, 2013",2018,5'9,150lbs,82+3,82+3,82+3,83+3,82+3,82+3,82+3,83+3,82+3,82+3,82+3,82+3,73+3,73+3,73+3,82+3,61+3,59+3,59+3,59+3,61+3,55+3,45+3,45+3,45+3,55+3,71,85,62,72,83,94,78,78,72,90,91,89,92,86,84,77,61,86,45,70,56,36,87,72,81,21,24,33,9,9,15,15,11,190871,2015
4,85,88,Right,2,5,4,High/ Medium,Normal,Yes,RCM,7,"Jan 18, 2014",2019,5'11,168lbs,80+0,80+0,80+0,83+0,83+0,83+0,83+0,83+0,85+0,85+0,85+0,84+0,80+0,80+0,80+0,84+0,67+0,70+0,70+0,70+0,67+0,61+0,55+0,55+0,55+0,61+0,80,82,53,87,81,85,76,77,82,86,77,78,79,85,79,84,65,85,75,86,63,52,83,88,75,37,39,40,15,13,5,10,13,192985,2015
5,88,90,Right,4,4,4,High/ Medium,Normal,Yes,LM,10,"Jul 1, 2012",2020,5'8,163lbs,82+2,82+2,82+2,87+2,84+2,84+2,84+2,87+2,87+2,87+2,87+2,86+2,78+2,78+2,78+2,86+2,62+2,64+2,64+2,64+2,62+2,54+2,46+2,46+2,46+2,54+2,78,83,57,86,79,93,82,79,82,90,93,87,92,86,90,79,59,74,63,82,54,41,84,86,86,25,27,22,11,12,6,8,8,183277,2015
6,87,87,Right,4,4,4,High/ Medium,Lean,Yes,RCM,19,"Aug 1, 2012",2018,5'9,143lbs,76+2,76+2,76+2,84+2,80+2,80+2,80+2,84+2,85+2,85+2,85+2,85+2,85+2,85+2,85+2,85+2,81+2,80+2,80+2,80+2,81+2,76+2,72+2,72+2,72+2,76+2,78,71,55,88,75,86,82,79,86,92,78,74,93,88,94,72,67,86,66,82,62,73,79,89,80,69,75,73,13,9,7,14,9,177003,2015
7,89,89,Right,5,4,4,High/ Medium,Normal,Yes,ST,9,"Jul 11, 2014",2019,6'0,187lbs,86+3,86+3,86+3,85+3,86+3,86+3,86+3,85+3,87+3,87+3,87+3,84+3,77+3,77+3,77+3,84+3,67+3,67+3,67+3,67+3,67+3,62+3,57+3,57+3,57+3,62+3,77,91,75,82,85,90,86,84,64,89,88,79,86,91,60,84,69,86,76,82,78,41,88,84,85,30,45,38,27,25,31,33,37,176580,2015
8,78,84,Right,1,3,3,High/ High,Normal,No,ST,21,"Jul 1, 2010",2020,6'2,143lbs,78+0,78+0,78+0,75+0,77+0,77+0,77+0,75+0,75+0,75+0,75+0,74+0,68+0,68+0,68+0,74+0,60+0,61+0,61+0,61+0,60+0,57+0,55+0,55+0,55+0,57+0,69,82,79,72,67,76,68,73,63,77,70,76,69,77,75,77,67,84,80,71,71,31,84,72,78,41,36,38,7,9,10,13,10,202126,2015
10,87,89,Right,4,4,4,High/ Medium,Normal,Yes,ST,9,"Jul 1, 2014",2019,6'1,174lbs,85+2,85+2,85+2,80+2,84+2,84+2,84+2,80+2,84+2,84+2,84+2,79+2,75+2,75+2,75+2,79+2,61+2,65+2,65+2,65+2,61+2,58+2,56+2,56+2,56+2,58+2,62,88,83,83,82,84,77,68,66,87,79,81,80,88,81,84,83,75,79,80,80,39,87,79,77,25,42,25,15,6,12,8,10,188545,2015
11,84,84,Right,3,3,2,Medium/ High,Lean,Yes,LCB,3,"Aug 1, 2010",2018,6'1,172lbs,60+1,60+1,60+1,60+1,60+1,60+1,60+1,60+1,60+1,60+1,60+1,63+1,66+1,66+1,66+1,63+1,75+1,76+1,76+1,76+1,75+1,79+1,83+1,83+1,83+1,79+1,55,42,87,73,47,53,49,51,69,67,69,72,62,81,58,67,85,73,78,43,84,86,48,52,50,86,84,87,6,8,15,5,15,182493,2015


In [66]:
detailed_data_2015_clean.to_csv('2015_data.csv', encoding='utf-8-sig')

In [50]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2014 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2014.ID = id_data.ID.values

In [51]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2014

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/14/157739'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2014.loc[detailed_data_2014.ID == id, key] = value

detailed_data_2014['Year'] = int(2014)

In [67]:
detailed_data_2014_clean = detailed_data_2014.drop('Loaned From', axis=1)

In [68]:
detailed_data_2014_clean = detailed_data_2014_clean.drop('Composure', axis=1)

In [69]:
detailed_data_2014_clean = detailed_data_2014_clean.dropna()

In [70]:
detailed_data_2014_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,94,97,Left,5,3,4,Medium/ Low,Normal,Yes,CF,10,"Jul 1, 2004",2016,5'7,148lbs,90+3,90+3,90+3,92+3,91+3,91+3,91+3,92+3,92+3,92+3,92+3,90+3,80+3,80+3,80+3,90+3,63+3,63+3,63+3,63+3,63+3,54+3,46+3,46+3,46+3,54+3,84,97,71,89,90,97,87,86,76,96,96,88,94,96,95,81,73,78,60,88,48,22,93,90,76,25,21,20,6,11,15,14,8,158023,2014
1,92,92,Right,5,4,5,High/ Low,Normal,Yes,LW,7,"Jul 1, 2009",2018,6'1,176lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,83,92,86,82,85,90,88,79,72,95,91,94,93,90,75,94,94,89,79,93,63,24,89,81,85,22,31,23,7,11,15,14,11,20801,2014
2,85,91,Right,4,5,5,High/ Medium,Lean,Yes,LW,10,"Jul 1, 2013",2018,5'9,141lbs,81+2,81+2,81+2,83+2,82+2,82+2,82+2,83+2,83+2,83+2,83+2,83+2,74+2,74+2,74+2,83+2,62+2,59+2,59+2,59+2,62+2,55+2,45+2,45+2,45+2,55+2,70,84,62,75,83,92,78,78,73,91,91,89,92,86,84,77,61,86,45,69,56,36,87,74,81,21,24,33,9,9,15,15,11,190871,2014
4,81,86,Right,2,5,4,High/ Medium,Normal,No,LCM,7,"Jan 18, 2014",2019,5'11,168lbs,76+0,76+0,76+0,80+0,79+0,79+0,79+0,80+0,81+0,81+0,81+0,81+0,77+0,77+0,77+0,81+0,65+0,66+0,66+0,66+0,65+0,58+0,51+0,51+0,51+0,58+0,78,78,41,82,78,82,76,76,82,82,75,82,78,78,77,82,65,85,74,83,42,52,82,80,75,37,39,40,15,13,5,10,13,192985,2014
5,88,90,Right,4,4,4,High/ Medium,Normal,Yes,LM,10,"Jul 1, 2012",2017,5'8,163lbs,81+2,81+2,81+2,86+2,84+2,84+2,84+2,86+2,86+2,86+2,86+2,86+2,78+2,78+2,78+2,86+2,62+2,64+2,64+2,64+2,62+2,54+2,46+2,46+2,46+2,54+2,82,83,57,86,79,90,82,79,82,89,91,87,92,84,90,79,59,73,63,82,54,41,84,85,85,25,27,22,11,12,6,8,8,183277,2014
6,86,88,Right,3,4,4,High/ Medium,Lean,Yes,RCM,19,"Aug 1, 2012",2017,5'9,143lbs,76+1,76+1,76+1,83+1,80+1,80+1,80+1,83+1,85+1,85+1,85+1,84+1,85+1,85+1,85+1,84+1,76+1,78+1,78+1,78+1,76+1,72+1,67+1,67+1,67+1,72+1,76,70,57,90,76,83,83,79,88,90,79,74,93,84,94,73,67,86,62,84,69,71,80,91,80,58,68,59,13,9,7,14,9,177003,2014
7,89,89,Right,5,4,4,High/ Medium,Normal,Yes,ST,7,"Jan 31, 2011",2018,5'11,179lbs,86+3,86+3,86+3,85+3,86+3,86+3,86+3,85+3,86+3,86+3,86+3,84+3,77+3,77+3,77+3,84+3,67+3,67+3,67+3,67+3,67+3,62+3,57+3,57+3,57+3,62+3,77,89,75,82,85,92,86,84,64,89,88,79,86,89,85,84,69,86,76,84,78,41,86,84,85,30,45,38,27,25,31,33,37,176580,2014
8,68,74,Right,1,3,3,Medium/ Medium,Normal,No,LS,37,"Jul 1, 2010",2017,6'2,143lbs,68+0,68+0,68+0,67+0,69+0,69+0,69+0,67+0,69+0,69+0,69+0,65+0,60+0,60+0,60+0,65+0,51+0,51+0,51+0,51+0,51+0,47+0,43+0,43+0,43+0,47+0,57,68,59,66,63,68,66,49,43,73,68,73,73,68,71,73,66,68,63,69,60,31,69,69,68,22,24,25,7,9,10,13,10,202126,2014
10,86,89,Right,4,4,4,High/ Medium,Normal,Yes,ST,9,"Jul 1, 2010",2014,6'0,172lbs,84+2,84+2,84+2,80+2,84+2,84+2,84+2,80+2,83+2,83+2,83+2,78+2,74+2,74+2,74+2,78+2,61+2,65+2,65+2,65+2,61+2,58+2,56+2,56+2,56+2,58+2,62,87,83,82,82,84,75,66,65,86,81,81,80,87,82,81,84,76,81,80,80,39,86,78,74,25,42,25,15,6,12,8,10,188545,2014
11,82,84,Right,3,3,2,Medium/ Medium,Lean,Yes,LCB,3,"Aug 1, 2010",2018,6'1,172lbs,60+1,60+1,60+1,60+1,60+1,60+1,60+1,60+1,61+1,61+1,61+1,63+1,67+1,67+1,67+1,63+1,74+1,76+1,76+1,76+1,74+1,78+1,81+1,81+1,81+1,78+1,55,42,84,73,47,53,49,51,69,66,64,68,62,80,58,67,80,76,77,43,83,85,48,58,50,83,83,86,6,8,15,5,15,182493,2014


In [71]:
detailed_data_2014_clean.to_csv('2014_data.csv', encoding='utf-8-sig')

In [56]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2013 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2013.ID = id_data.ID.values

In [57]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2013

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/13/157375'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2013.loc[detailed_data_2013.ID == id, key] = value

detailed_data_2013['Year'] = int(2013)

In [72]:
detailed_data_2013_clean = detailed_data_2013.drop('Loaned From', axis=1)

In [73]:
detailed_data_2013_clean = detailed_data_2013_clean.drop('Composure', axis=1)

In [74]:
detailed_data_2013_clean = detailed_data_2013_clean.dropna()

In [75]:
detailed_data_2013_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,94,97,Left,5,3,4,High/ Medium,Normal,Yes,RW,10,"Jul 1, 2004",2016,5'7,148lbs,90+3,90+3,90+3,92+3,91+3,91+3,91+3,92+3,92+3,92+3,92+3,90+3,80+3,80+3,80+3,90+3,62+3,62+3,62+3,62+3,62+3,53+3,44+3,44+3,44+3,53+3,84,97,72,89,89,97,87,86,76,96,95,91,94,96,95,80,73,80,65,88,48,22,93,90,78,13,21,20,6,11,15,14,8,158023,2013
1,92,95,Right,5,4,5,High/ Low,Normal,Yes,LW,7,"Jul 1, 2009",2015,6'1,185lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,83,92,86,82,85,90,88,79,72,95,91,94,93,90,75,94,94,89,79,93,63,24,89,81,85,22,31,23,7,11,15,14,11,20801,2013
2,83,91,Right,3,5,5,High/ Medium,Lean,Yes,LW,10,"May 28, 2013",2016,5'9,143lbs,82+1,82+1,82+1,84+1,83+1,83+1,83+1,84+1,84+1,84+1,84+1,83+1,75+1,75+1,75+1,83+1,62+1,60+1,60+1,60+1,62+1,55+1,45+1,45+1,45+1,55+1,70,84,62,79,83,92,78,78,75,92,92,89,93,84,84,77,61,86,46,75,56,36,88,74,81,21,24,33,9,9,15,15,11,190871,2013
4,79,85,Right,2,5,4,High/ Medium,Normal,No,RW,7,"Jan 1, 2012",2017,5'11,168lbs,74+0,74+0,74+0,79+0,77+0,77+0,77+0,79+0,79+0,79+0,79+0,80+0,76+0,76+0,76+0,80+0,66+0,67+0,67+0,67+0,66+0,58+0,50+0,50+0,50+0,58+0,78,78,35,80,78,82,76,76,80,81,74,82,75,78,75,80,65,85,73,83,39,59,77,80,75,37,38,39,15,13,5,10,13,192985,2013
5,85,90,Right,3,4,4,High/ Low,Normal,No,LM,10,"Jul 1, 2012",2017,5'8,159lbs,80+1,80+1,80+1,85+1,82+1,82+1,82+1,85+1,85+1,85+1,85+1,84+1,77+1,77+1,77+1,84+1,62+1,64+1,64+1,64+1,62+1,54+1,47+1,47+1,47+1,54+1,80,78,64,85,79,88,82,79,82,87,91,86,92,84,90,76,59,72,63,80,54,41,83,84,85,25,27,22,11,12,6,8,8,183277,2013
6,84,88,Right,3,4,4,High/ Medium,Lean,Yes,CM,19,"Aug 1, 2012",2017,5'9,143lbs,75+1,75+1,75+1,82+1,79+1,79+1,79+1,82+1,83+1,83+1,83+1,83+1,83+1,83+1,83+1,83+1,76+1,78+1,78+1,78+1,76+1,72+1,66+1,66+1,66+1,72+1,76,70,57,90,76,82,83,79,88,85,79,74,92,84,94,70,67,86,62,83,69,71,80,85,80,58,68,59,13,9,7,14,9,177003,2013
7,86,87,Right,4,4,4,High/ Medium,Normal,Yes,CF,9,"Jan 31, 2011",2016,5'11,179lbs,83+2,83+2,83+2,84+2,84+2,84+2,84+2,84+2,85+2,85+2,85+2,83+2,76+2,76+2,76+2,83+2,66+2,66+2,66+2,66+2,66+2,61+2,57+2,57+2,57+2,61+2,77,84,72,82,81,91,84,84,64,88,86,78,86,88,85,82,69,86,76,81,78,41,86,81,85,30,45,38,27,25,31,33,37,176580,2013
8,67,80,Right,1,3,3,Medium/ Medium,Normal,No,CF,37,"Jul 1, 2010",2017,6'0,143lbs,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,68+0,68+0,68+0,64+0,58+0,58+0,58+0,64+0,49+0,50+0,50+0,50+0,49+0,45+0,42+0,42+0,42+0,45+0,57,69,59,63,63,61,66,49,41,74,73,71,73,68,71,68,66,62,61,65,60,31,67,69,68,22,24,17,7,9,10,13,10,202126,2013
10,85,88,Right,3,4,3,High/ Medium,Normal,Yes,ST,9,"Jul 1, 2010",2014,6'0,172lbs,84+1,84+1,84+1,78+1,82+1,82+1,82+1,78+1,80+1,80+1,80+1,75+1,69+1,69+1,69+1,75+1,58+1,59+1,59+1,59+1,58+1,55+1,53+1,53+1,53+1,55+1,59,85,83,80,83,82,67,33,43,86,82,82,80,87,78,81,85,74,79,76,78,38,86,65,73,20,34,23,15,6,12,8,10,188545,2013
11,82,84,Right,2,3,2,Medium/ Medium,Normal,No,LCB,3,"Jan 1, 2010",2015,6'1,165lbs,60+0,60+0,60+0,60+0,60+0,60+0,60+0,60+0,61+0,61+0,61+0,63+0,67+0,67+0,67+0,63+0,74+0,76+0,76+0,76+0,74+0,79+0,82+0,82+0,82+0,79+0,55,42,84,73,47,53,49,51,69,66,64,68,62,80,58,67,81,76,78,43,83,87,48,58,50,85,84,87,6,8,15,5,15,182493,2013


In [78]:
detailed_data_2013_clean.to_csv('2013_data.csv', encoding='utf-8-sig')

In [76]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2012 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2012.ID = id_data.ID.values

In [77]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2012

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/13/157011'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2012.loc[detailed_data_2012.ID == id, key] = value

detailed_data_2012['Year'] = int(2012)

In [79]:
detailed_data_2012_clean = detailed_data_2012.drop('Loaned From', axis=1)

In [81]:
detailed_data_2012_clean = detailed_data_2012_clean.drop('Composure', axis=1)

In [86]:
detailed_data_2012_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,94,96,Left,5,3,4,High/ Medium,Normal,Yes,CF,10,"Jul 1, 2004",2016,5'7,148lbs,90+3,90+3,90+3,92+3,91+3,91+3,91+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,53+3,45+3,45+3,45+3,53+3,85,95,73,89,89,97,87,82,75,96,95,92,94,95,95,80,73,80,67,88,48,22,92,90,76,13,21,20,6,11,15,14,8,158023,2012
1,92,94,Right,5,4,5,High/ Low,Normal,Yes,LW,7,"Jul 1, 2009",2015,6'1,185lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,84,91,87,82,85,91,88,79,72,95,90,94,93,88,75,94,94,87,79,92,63,24,88,80,85,22,31,23,7,11,15,14,11,20801,2012
2,85,91,Right,3,4,5,Medium/ Medium,Lean,Yes,LW,11,"Jan 1, 2009",2014,5'9,143lbs,82+1,82+1,82+1,84+1,84+1,84+1,84+1,84+1,84+1,84+1,84+1,83+1,76+1,76+1,76+1,83+1,62+1,61+1,61+1,61+1,62+1,56+1,46+1,46+1,46+1,56+1,70,84,65,76,80,90,76,76,74,92,92,90,95,84,84,78,61,86,46,76,56,36,88,81,77,21,32,33,9,9,15,15,11,190871,2012
5,87,92,Right,3,4,4,High/ Medium,Normal,No,RW,10,"Jul 1, 2012",2017,5'8,159lbs,80+1,80+1,80+1,86+1,82+1,82+1,82+1,86+1,85+1,85+1,85+1,85+1,77+1,77+1,77+1,85+1,62+1,63+1,63+1,63+1,62+1,54+1,47+1,47+1,47+1,54+1,80,79,64,85,79,90,82,79,82,88,91,90,92,82,90,76,59,72,63,78,54,41,84,84,85,25,27,22,11,12,6,8,8,183277,2012
6,86,89,Right,3,4,4,High/ Medium,Lean,Yes,CAM,19,"Aug 1, 2012",2017,5'9,148lbs,76+1,76+1,76+1,84+1,80+1,80+1,80+1,84+1,85+1,85+1,85+1,85+1,83+1,83+1,83+1,85+1,73+1,76+1,76+1,76+1,73+1,68+1,62+1,62+1,62+1,68+1,78,72,57,90,78,84,83,79,88,90,82,76,93,89,94,68,67,83,61,71,69,57,83,91,80,58,55,49,13,9,7,14,9,177003,2012
7,86,88,Right,4,4,4,High/ Medium,Normal,Yes,ST,9,"Jan 1, 2011",2016,5'11,179lbs,83+2,83+2,83+2,84+2,84+2,84+2,84+2,84+2,85+2,85+2,85+2,82+2,76+2,76+2,76+2,82+2,66+2,66+2,66+2,66+2,66+2,61+2,56+2,56+2,56+2,61+2,77,85,68,82,81,90,82,83,61,88,86,78,83,85,83,82,69,84,76,81,76,41,86,81,85,30,45,38,27,25,31,33,37,176580,2012
10,83,86,Right,3,4,3,High/ High,Normal,Yes,ST,9,"Jul 1, 2010",2014,6'0,172lbs,82+1,82+1,82+1,78+1,81+1,81+1,81+1,78+1,79+1,79+1,79+1,74+1,68+1,68+1,68+1,74+1,57+1,58+1,58+1,58+1,57+1,54+1,52+1,52+1,52+1,54+1,59,83,82,78,78,83,67,33,43,86,82,82,80,84,77,81,85,74,78,72,75,34,86,65,66,18,34,23,15,6,12,8,10,188545,2012
11,79,82,Right,2,3,2,Medium/ Medium,Normal,No,LCB,3,"Jan 1, 2010",2015,6'1,165lbs,59+0,59+0,59+0,59+0,59+0,59+0,59+0,59+0,60+0,60+0,60+0,62+0,65+0,65+0,65+0,62+0,71+0,73+0,73+0,73+0,71+0,75+0,79+0,79+0,79+0,75+0,55,42,83,73,51,53,49,51,68,64,64,68,62,72,58,67,81,71,77,43,80,78,48,58,50,81,77,86,6,8,15,5,15,182493,2012
12,88,91,Left,3,2,4,High/ Low,Normal,Yes,RW,21,"Jan 1, 2010",2014,5'7,148lbs,79+1,79+1,79+1,85+1,82+1,82+1,82+1,85+1,87+1,87+1,87+1,85+1,79+1,79+1,79+1,85+1,62+1,65+1,65+1,65+1,62+1,55+1,47+1,47+1,47+1,55+1,81,78,58,90,81,86,83,77,85,90,83,74,93,85,88,78,66,68,55,81,51,41,84,91,77,23,30,29,13,9,13,9,13,168542,2012
13,87,89,Right,4,3,3,High/ Medium,Normal,Yes,LCB,15,"Jan 1, 2005",2013,6'0,179lbs,69+2,69+2,69+2,72+2,69+2,69+2,69+2,72+2,69+2,69+2,69+2,73+2,73+2,73+2,73+2,73+2,82+2,80+2,80+2,80+2,82+2,84+2,85+2,85+2,85+2,84+2,74,59,89,78,55,66,73,62,70,83,79,79,84,78,60,71,93,86,83,55,85,87,52,63,66,84,86,89,11,8,9,7,11,155862,2012


In [85]:
detailed_data_2012_clean = detailed_data_2012_clean.dropna()

In [87]:
detailed_data_2012_clean.to_csv('2012_data.csv', encoding='utf-8-sig')

In [88]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2011 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2011.ID = id_data.ID.values

In [89]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2011

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/12/156644'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2011.loc[detailed_data_2011.ID == id, key] = value

detailed_data_2011['Year'] = int(2011)

In [90]:
detailed_data_2011_clean = detailed_data_2011.drop('Loaned From', axis=1)

In [91]:
detailed_data_2011_clean = detailed_data_2011_clean.drop('Composure', axis=1)

In [94]:
detailed_data_2011_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,94,96,Left,5,3,4,High/ Medium,Normal,Yes,RW,10,"Jan 1, 2004",2016,5'7,148lbs,88+3,88+3,88+3,92+3,91+3,91+3,91+3,92+3,92+3,92+3,92+3,90+3,80+3,80+3,80+3,90+3,62+3,61+3,61+3,61+3,62+3,53+3,42+3,42+3,42+3,53+3,85,92,69,89,91,97,87,82,75,97,95,92,94,95,95,81,73,83,48,88,48,22,92,90,86,13,21,20,6,11,15,14,8,158023,2011
1,92,94,Right,5,4,5,High/ Low,Normal,Yes,LW,7,"Jan 1, 2009",2015,6'1,185lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,84,90,87,83,85,93,88,87,72,95,90,94,93,88,73,93,94,87,79,90,63,24,88,81,87,22,31,23,7,11,15,14,11,20801,2011
2,84,91,Right,3,4,5,Medium/ Medium,Lean,No,LW,11,"Jan 1, 2009",2015,5'8,132lbs,81+1,81+1,81+1,84+1,83+1,83+1,83+1,84+1,84+1,84+1,84+1,82+1,74+1,74+1,74+1,82+1,59+1,58+1,58+1,58+1,59+1,53+1,45+1,45+1,45+1,53+1,73,82,66,77,80,90,75,73,76,91,93,86,94,84,88,78,61,46,33,76,56,36,88,82,77,21,32,33,9,9,15,15,11,190871,2011
4,75,83,Right,1,5,3,Medium/ Medium,Lean,No,CAM,21,"Jan 1, 2008",2015,5'11,150lbs,68+0,68+0,68+0,75+0,72+0,72+0,72+0,75+0,75+0,75+0,75+0,75+0,71+0,71+0,71+0,75+0,56+0,58+0,58+0,58+0,56+0,48+0,39+0,39+0,39+0,48+0,74,71,35,77,69,75,76,75,82,77,67,78,74,75,73,67,64,69,54,79,35,45,70,80,63,19,21,23,15,13,5,10,13,192985,2011
5,85,89,Right,3,4,4,High/ Medium,Normal,No,RW,7,"Jan 1, 2007",2015,5'7,152lbs,80+1,80+1,80+1,84+1,82+1,82+1,82+1,84+1,84+1,84+1,84+1,83+1,75+1,75+1,75+1,83+1,61+1,62+1,62+1,62+1,61+1,54+1,46+1,46+1,46+1,54+1,75,76,64,83,80,90,82,81,71,89,93,88,92,82,90,76,59,72,63,79,54,41,82,82,82,25,27,22,11,12,6,8,8,183277,2011
6,88,89,Right,3,5,4,High/ Medium,Lean,Yes,LCM,10,"Jan 1, 2008",2016,5'9,148lbs,78+1,78+1,78+1,85+1,81+1,81+1,81+1,85+1,87+1,87+1,87+1,86+1,83+1,83+1,83+1,86+1,74+1,76+1,76+1,76+1,74+1,68+1,63+1,63+1,63+1,68+1,79,77,57,90,80,84,83,82,88,91,80,78,95,89,94,70,67,87,64,76,69,55,86,92,80,58,55,49,13,9,7,14,9,177003,2011
7,85,88,Right,4,4,4,High/ Medium,Normal,No,RS,9,"Jan 1, 2011",2016,5'11,179lbs,82+2,82+2,82+2,83+2,83+2,83+2,83+2,83+2,83+2,83+2,83+2,81+2,75+2,75+2,75+2,81+2,65+2,65+2,65+2,65+2,65+2,61+2,56+2,56+2,56+2,61+2,80,86,67,81,81,86,82,83,61,85,83,78,79,86,79,82,69,84,77,81,76,41,85,81,85,30,45,38,27,25,31,33,37,176580,2011
8,62,72,Right,1,3,3,Medium/ Medium,Normal,No,CF,37,"Jan 1, 2009",2012,6'0,143lbs,62+0,62+0,62+0,59+0,62+0,62+0,62+0,59+0,61+0,61+0,61+0,57+0,52+0,52+0,52+0,57+0,44+0,45+0,45+0,45+0,44+0,42+0,39+0,39+0,39+0,42+0,37,64,59,54,45,61,66,49,32,62,74,71,75,67,71,63,66,62,47,51,63,31,61,62,62,22,24,17,7,9,10,13,10,202126,2011
10,77,81,Right,2,4,3,Medium/ Medium,Normal,Yes,ST,9,"Jan 1, 2010",2014,6'0,172lbs,77+0,77+0,77+0,73+0,76+0,76+0,76+0,73+0,75+0,75+0,75+0,71+0,65+0,65+0,65+0,71+0,54+0,55+0,55+0,55+0,54+0,50+0,47+0,47+0,47+0,50+0,59,75,78,75,75,76,67,33,43,78,79,80,71,75,74,81,82,74,75,74,59,35,77,68,70,13,35,18,15,6,12,8,10,188545,2011
11,80,82,Right,2,3,2,Medium/ Medium,Lean,No,CB,3,"Jan 1, 2010",2015,6'1,165lbs,60+0,60+0,60+0,61+0,61+0,61+0,61+0,61+0,62+0,62+0,62+0,65+0,67+0,67+0,67+0,65+0,74+0,75+0,75+0,75+0,74+0,78+0,80+0,80+0,80+0,78+0,55,42,83,73,51,59,49,51,68,71,64,68,63,79,58,67,81,77,77,43,80,82,48,58,50,81,80,86,6,8,15,5,15,182493,2011


In [93]:
detailed_data_2011_clean = detailed_data_2011_clean.dropna()

In [95]:
detailed_data_2011_clean.to_csv('2011_data.csv', encoding='utf-8-sig')

In [96]:
# Get detailed player information from player page
detailed_columns = ['Overall', 'Expert Potential', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']
detailed_data_2010 = pd.DataFrame(index = range(0, id_data.count()[0]), columns = detailed_columns)
detailed_data_2010.ID = id_data.ID.values

In [97]:
player_data_url = 'https://sofifa.com/player/'
# Generate the data for 2010

for id in id_data.ID[:2000]:
    url = player_data_url + str(id) + '/11/156279'
    source_code = requests.get(url)
    redirect_url = source_code.url
    if redirect_url == url:
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        skill_map = {}
        overall = soup.find('div', {'class': 'column col-4 text-center'}).find('span').text
        skill_map['Overall'] = int(overall)
        first_find = soup.find('div', {'class': 'column col-4 text-center'})
        second_find = first_find.find_next('div').find('span').text
        skill_map['Expert Potential'] = int(second_find)
        columns = soup.find('div', {'class': 'teams'}).find('div', {'class': 'columns'}).find_all('div', {'class': 'column col-4'})
        for column in columns:
            skills = column.find_all('li')
            for skill in skills:
                if(skill.find('label') != None):
                    label = skill.find('label').text
                    value = skill.text.replace(label, '').strip()
                    skill_map[label] = value
        meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
        length = len(meta_data)
        weight = meta_data[length - 1]
        height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
        skill_map["Height"] = height
        skill_map['Weight'] = weight
        if('Position' in skill_map.keys()):
            if skill_map['Position'] in ('', 'RES', 'SUB'):
                skill_map['Position'] = soup.find('article').find('div', {'class': 'meta'}).find('span').text
            if(skill_map['Position'] != 'GK'):
                card_rows = soup.find('aside').find('div', {'class': 'card mb-2'}).find('div', {'class': 'card-body'}).findAll('div', {'class': 'columns'})
                for c_row in card_rows:
                    attributes = c_row.find_all('div', {'class': re.compile('column col-sm-2 text-center')})
                    for attribute in attributes:
                        if(attribute.find('div')):
                            name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                            value = attribute.text.replace(name, '').strip()
                            skill_map[str(name)] = value
        sections = soup.find('article').find_all('div', {'class': 'mb-2'})[1:3]
        first = sections[0].find_all('div', {'class': 'column col-4'})
        second = sections[1].find_all('div', {'class': 'column col-4'})[:-1]
        sections = first + second
        for section in sections:
            items = section.find('ul').find_all('li')
            for item in items:
                value = int(re.findall(r'\d+', item.text)[0])
                name = ''.join(re.findall('[a-zA-Z]*', item.text))
                skill_map[str(name)] = value
        for key, value in skill_map.items():
            detailed_data_2010.loc[detailed_data_2010.ID == id, key] = value

detailed_data_2010['Year'] = int(2010)

In [98]:
detailed_data_2010

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,90,95,Left,5,3,5,NaN,Normal,Yes,RW,10,"Jan 1, 2000",NaN,2016,5'7,148lbs,87+3,87+3,87+3,90+3,88+3,88+3,88+3,90+3,89+3,89+3,89+3,90+3,81+3,81+3,81+3,90+3,60+3,59+3,59+3,59+3,60+3,51+3,45+3,45+3,45+3,51+3,85,91,69,89,88,97,87,82,75,96,94,88,96,89,86,79,64,75,59,87,48,22,90,91,86,NaN,13,21,20,6,11,15,14,8,158023,2010
1,89,94,Right,5,4,5,NaN,Normal,Yes,LW,7,"Jan 1, 2009",NaN,2015,6'1,165lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,63+3,63+3,63+3,63+3,63+3,57+3,55+3,55+3,55+3,57+3,84,89,87,81,85,93,88,88,71,95,93,94,94,88,84,93,91,85,80,91,61,24,88,80,82,NaN,22,31,23,7,11,15,14,11,20801,2010
2,78,88,Right,3,4,5,NaN,Lean,No,ST,21,"Jan 1, 2009",NaN,2015,5'8,132lbs,79+1,79+1,79+1,83+1,80+1,80+1,80+1,83+1,80+1,80+1,80+1,82+1,74+1,74+1,74+1,82+1,59+1,58+1,58+1,58+1,59+1,53+1,49+1,49+1,49+1,53+1,73,80,66,77,79,93,74,68,71,84,90,86,94,89,77,73,65,62,44,69,64,36,80,81,83,NaN,21,32,33,9,9,15,15,11,190871,2010
3,66,81,Right,1,2,1,NaN,Normal,No,GK,32,"Sep 2, 2010",NaN,2012,6'1,170lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,5,9,18,7,8,7,8,25,19,25,29,44,61,49,25,68,48,67,6,45,18,10,23,10,NaN,13,11,17,62,69,57,71,65,200389,2010
4,70,77,Right,1,4,3,NaN,Normal,No,CAM,27,"Jan 1, 2005",NaN,2015,5'11,150lbs,68+0,68+0,68+0,70+0,69+0,69+0,69+0,70+0,70+0,70+0,70+0,70+0,67+0,67+0,67+0,70+0,52+0,54+0,54+0,54+0,52+0,47+0,44+0,44+0,44+0,47+0,69,68,62,67,61,71,70,71,73,74,69,69,66,63,59,67,61,67,63,76,55,47,67,71,63,NaN,19,21,23,15,13,5,10,13,192985,2010
5,81,91,Right,1,3,4,NaN,Normal,No,RW,7,"Jan 1, 2006",NaN,2014,5'7,152lbs,77+0,77+0,77+0,81+0,78+0,78+0,78+0,81+0,79+0,79+0,79+0,81+0,75+0,75+0,75+0,81+0,61+0,61+0,61+0,61+0,61+0,55+0,51+0,51+0,51+0,55+0,74,76,64,80,67,84,68,64,68,86,87,86,83,78,76,72,67,74,56,74,54,42,80,82,75,NaN,35,32,29,11,12,6,8,8,183277,2010
6,83,90,Right,2,5,4,NaN,Lean,Yes,LCM,14,"Jan 1, 2008",NaN,2016,5'8,143lbs,77+0,77+0,77+0,84+0,79+0,79+0,79+0,84+0,84+0,84+0,84+0,85+0,82+0,82+0,82+0,85+0,70+0,71+0,71+0,71+0,70+0,66+0,63+0,63+0,63+0,66+0,78,77,57,89,77,87,83,82,87,88,77,80,90,86,75,70,66,87,67,73,69,51,83,91,80,NaN,58,50,44,13,9,7,14,9,177003,2010
7,82,88,Right,3,4,4,NaN,Normal,No,RF,9,"Jan 1, 2007",NaN,2013,5'11,179lbs,83+1,83+1,83+1,82+1,83+1,83+1,83+1,82+1,82+1,82+1,82+1,81+1,77+1,77+1,77+1,81+1,64+1,65+1,65+1,65+1,64+1,61+1,59+1,59+1,59+1,61+1,80,85,69,81,86,85,82,81,61,84,81,78,79,86,84,85,71,82,78,84,79,41,85,84,88,NaN,30,45,38,27,25,31,33,37,176580,2010
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202126,2010
9,78,85,Right,2,3,1,NaN,Lean,No,GK,13,"Jan 1, 2009",NaN,2013,6'4,181lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,13,9,11,8,8,10,19,9,18,51,55,52,68,42,55,64,40,48,12,28,23,18,58,21,NaN,9,7,8,78,84,84,74,79,193080,2010


In [101]:
detailed_data_2010_clean = detailed_data_2010.drop('Loaned From', axis=1)

In [102]:
detailed_data_2010_clean = detailed_data_2010_clean.drop('Composure', axis=1)

In [103]:
detailed_data_2010_clean = detailed_data_2010_clean.drop('Work Rate', axis=1)

In [106]:
detailed_data_2010_clean

,Overall,Expert Potential,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,ID,Year
0,90,95,Left,5,3,5,Normal,Yes,RW,10,"Jan 1, 2000",2016,5'7,148lbs,87+3,87+3,87+3,90+3,88+3,88+3,88+3,90+3,89+3,89+3,89+3,90+3,81+3,81+3,81+3,90+3,60+3,59+3,59+3,59+3,60+3,51+3,45+3,45+3,45+3,51+3,85,91,69,89,88,97,87,82,75,96,94,88,96,89,86,79,64,75,59,87,48,22,90,91,86,13,21,20,6,11,15,14,8,158023,2010
1,89,94,Right,5,4,5,Normal,Yes,LW,7,"Jan 1, 2009",2015,6'1,165lbs,90+3,90+3,90+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,63+3,63+3,63+3,63+3,63+3,57+3,55+3,55+3,55+3,57+3,84,89,87,81,85,93,88,88,71,95,93,94,94,88,84,93,91,85,80,91,61,24,88,80,82,22,31,23,7,11,15,14,11,20801,2010
2,78,88,Right,3,4,5,Lean,No,ST,21,"Jan 1, 2009",2015,5'8,132lbs,79+1,79+1,79+1,83+1,80+1,80+1,80+1,83+1,80+1,80+1,80+1,82+1,74+1,74+1,74+1,82+1,59+1,58+1,58+1,58+1,59+1,53+1,49+1,49+1,49+1,53+1,73,80,66,77,79,93,74,68,71,84,90,86,94,89,77,73,65,62,44,69,64,36,80,81,83,21,32,33,9,9,15,15,11,190871,2010
4,70,77,Right,1,4,3,Normal,No,CAM,27,"Jan 1, 2005",2015,5'11,150lbs,68+0,68+0,68+0,70+0,69+0,69+0,69+0,70+0,70+0,70+0,70+0,70+0,67+0,67+0,67+0,70+0,52+0,54+0,54+0,54+0,52+0,47+0,44+0,44+0,44+0,47+0,69,68,62,67,61,71,70,71,73,74,69,69,66,63,59,67,61,67,63,76,55,47,67,71,63,19,21,23,15,13,5,10,13,192985,2010
5,81,91,Right,1,3,4,Normal,No,RW,7,"Jan 1, 2006",2014,5'7,152lbs,77+0,77+0,77+0,81+0,78+0,78+0,78+0,81+0,79+0,79+0,79+0,81+0,75+0,75+0,75+0,81+0,61+0,61+0,61+0,61+0,61+0,55+0,51+0,51+0,51+0,55+0,74,76,64,80,67,84,68,64,68,86,87,86,83,78,76,72,67,74,56,74,54,42,80,82,75,35,32,29,11,12,6,8,8,183277,2010
6,83,90,Right,2,5,4,Lean,Yes,LCM,14,"Jan 1, 2008",2016,5'8,143lbs,77+0,77+0,77+0,84+0,79+0,79+0,79+0,84+0,84+0,84+0,84+0,85+0,82+0,82+0,82+0,85+0,70+0,71+0,71+0,71+0,70+0,66+0,63+0,63+0,63+0,66+0,78,77,57,89,77,87,83,82,87,88,77,80,90,86,75,70,66,87,67,73,69,51,83,91,80,58,50,44,13,9,7,14,9,177003,2010
7,82,88,Right,3,4,4,Normal,No,RF,9,"Jan 1, 2007",2013,5'11,179lbs,83+1,83+1,83+1,82+1,83+1,83+1,83+1,82+1,82+1,82+1,82+1,81+1,77+1,77+1,77+1,81+1,64+1,65+1,65+1,65+1,64+1,61+1,59+1,59+1,59+1,61+1,80,85,69,81,86,85,82,81,61,84,81,78,79,86,84,85,71,82,78,84,79,41,85,84,88,30,45,38,27,25,31,33,37,176580,2010
10,76,85,Right,3,4,3,Normal,Yes,LS,9,"Sep 2, 2010",2014,6'0,168lbs,77+1,77+1,77+1,71+1,74+1,74+1,74+1,71+1,72+1,72+1,72+1,69+1,66+1,66+1,66+1,69+1,53+1,54+1,54+1,54+1,53+1,49+1,48+1,48+1,48+1,49+1,52,82,75,68,75,74,67,33,40,77,81,77,76,76,70,77,74,71,72,68,56,35,76,70,71,13,35,18,15,6,12,8,10,188545,2010
11,81,82,Right,2,3,2,Lean,No,LCB,3,"Sep 2, 2010",2015,6'1,165lbs,63+0,63+0,63+0,64+0,62+0,62+0,62+0,64+0,61+0,61+0,61+0,64+0,66+0,66+0,66+0,64+0,76+0,76+0,76+0,76+0,76+0,79+0,81+0,81+0,81+0,79+0,55,42,85,70,51,54,49,51,61,62,74,76,65,81,80,67,83,79,77,43,80,84,48,58,50,83,82,87,6,8,15,5,15,182493,2010
12,85,90,Left,3,3,4,Normal,Yes,CAM,21,"Sep 2, 2010",2014,5'7,148lbs,81+1,81+1,81+1,86+1,83+1,83+1,83+1,86+1,85+1,85+1,85+1,86+1,79+1,79+1,79+1,86+1,62+1,62+1,62+1,62+1,62+1,54+1,49+1,49+1,49+1,54+1,84,79,58,90,81,89,82,77,73,91,87,83,87,85,80,79,60,70,59,85,57,41,83,90,77,23,30,29,13,9,13,9,13,168542,2010


In [105]:
detailed_data_2010_clean = detailed_data_2010_clean.dropna()

In [107]:
detailed_data_2010_clean.to_csv('2010_data.csv', encoding='utf-8-sig')